# Default settings

In [2]:
# import dependencies

import os
import sys
import gzip
import shutil
import decimal
import pathlib
import operator
import subprocess
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from time import sleep
from nilearn.image import iter_img
from nilearn.decomposition import CanICA
from nilearn.masking import apply_mask
from nilearn.plotting import plot_stat_map, show
from scipy.interpolate import RegularGridInterpolator

/home/john/.local/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [ ]:
# config defaults
default_config = { 
    "SKULLSTRIP" : "1",
    "SLICETIME"  : "1",
    "MOTCOR"     : "1",
    "NORM"       : "1",
    "SMOOTH"     : "6", # gaussian smoothing kernel size in mm
    "MOTREG"     : "1",
    "GSR"        : "1",
    "NUISANCE"   : "3",
    "TEMPLATE"   : "/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz",
    "SCRUB"      : "UNION" # union, intersect, dvars, fd, none
}

In [ ]:
step_order={
    "BASELINE"  :0,
    "SKULLSTRIP":1,
    "SLICETIME" :2,
    "MOTCOR"    :3,
    "NORM"      :4,
    "GSR"       :5,
    "MOTREG"    :6,
    "NUISANCE"  :7,
    "SCRUB"     :8,
    "SMOOTH"    :9
}

# Support functions

In [ ]:
def create_dirstruct(output):
    pathlib.Path(os.path.join(output, "func"           )).mkdir(parents=True, exist_ok=True)
    pathlib.Path(os.path.join(output, "motion"         )).mkdir(parents=True, exist_ok=True)
    pathlib.Path(os.path.join(output, "spat_norm"      )).mkdir(parents=True, exist_ok=True)
    pathlib.Path(os.path.join(output, "quality_control")).mkdir(parents=True, exist_ok=True)
    pathlib.Path(os.path.join(output, "anat", "segment")).mkdir(parents=True, exist_ok=True)

    return

In [ ]:
def check_exist(filepath, type="any"):
    if type in ["file", "f"]:
        from os.path import isfile as exists
    elif type=="dir" or type=="d":
        from os.path import isdir as exists
    elif type=="any":
        from os.path import exists
    else:
        raise Exception("type '" + type + "' is invalid.")
    
    return(exists(filepath))

In [ ]:
def parse_line(text, keywords, split="="):
    key, val = text.split("=")
    if key not in keywords:
        raise Exception("Input doesn't contain any of the specified key words")
    return(key, val)

In [ ]:
def strip_chars(text, first, last):
    text = "{}".format(text[1:] if text.startswith(first) else text)
    text = "{}".format(text[:-1] if text.endswith(last) else text)
    return(text)

In [ ]:
def parse_input(text, keywords, split="=", first="[", last="]"):
    key, val = parse_line(text, keywords, split)
    val = strip_chars(val, "[", "]")
    return(key, val)

In [ ]:
def check_defaults(config_dict, defaults):
    default_keys=[key for key in defaults.keys()]

    restore_defaults=[key not in config_dict.keys() for key in default_keys]
    restore_defaults=list(filter(lambda x: restore_defaults[x], range(len(restore_defaults))))
    restore_defaults=[default_keys[i] for i in restore_defaults]

    for key in restore_defaults:
        config_dict[key] = defaults[key]

    return(config_dict)

In [ ]:
def voxel_size(img, excl_time=True):
    nii = nib.load(img)
    vox_sz = nii.header.get_zooms()

    if excl_time:
        vox_sz=vox_sz[0:3]
    return(vox_sz)

In [ ]:
def interpret_input_line(line, keywords):
    input_dict = {}
    items = line.split(",")
    
    for item in items:
        item=item.strip()
        try:
            key, val = parse_input(item, keywords) # keywords must be enterest as a list
            input_dict[key] = val
        except:
            print("Error:", item)

    # check if all files exist 
    if not check_exist(input_dict["FUNC"], "file"):
        raise Exception(input_dict["FUNC"] + " is not a valid filepath.")
    if not check_exist(input_dict["ANAT"], "file"):
        raise Exception(input_dict["ANAT"] + " is not a valid filepath.")
       
    return(input_dict)

In [ ]:
def interpret_input(filepath):
    input = open(filepath, "r")
    lines = input.readlines()

    keywords = ["FUNC", "ANAT", "OUTPUT"]
    input_df = pd.DataFrame(index=range(len(lines)), columns=keywords)

    for line, ind in zip(lines, range(len(lines))):
        line=line.strip()
        input_dict=interpret_input_line(line, keywords)

        input_df.loc[ind,"FUNC"  ] = input_dict["FUNC"]
        input_df.loc[ind,"ANAT"  ] = input_dict["ANAT"]
        input_df.loc[ind,"OUTPUT"] = input_dict["OUTPUT"]

    return(input_df)
    

In [ ]:
def interpret_config(filepath):
    config=open(filepath, "r")
    lines =config.readlines()
    config_dict = {}

    for line in lines:
        line=line.strip()
        try:
            key, val = parse_input(line, default_config.keys) # keywords must be enterest as a list
            config_dict[key] = val
        except:
            print("Error:", line)

    config_dict = check_defaults(config_dict, default_config)
    return(config_dict)

In [ ]:
def afni2nifti(filepath, rm_orig=True):    
    nii_filepath=os.path.join(os.path.dirname(filepath), rm_ext(filepath))+".nii"
    os.system("3dAFNItoNIFTI -prefix {} {}".format(nii_filepath, filepath+"*.HEAD"))
    if rm_orig:
        os.system("rm {}*.HEAD".format(filepath))
        os.system("rm {}*.BRIK".format(filepath))
    return(nii_filepath)

In [ ]:
def getTR(filepath):    
    img = nib.load(filepath)
    tr  = img.header.get_zooms()[3]
    return(str(tr)) 

In [ ]:
def rm_ext(filename):
    filename=os.path.basename(filename)
    newpath =os.path.splitext(filename)[0]
    
    if "." in newpath:
        newpath=rm_ext(newpath)
    return(newpath)  

In [ ]:
def get_ext(filename):
    filename    =os.path.basename(filename)
    newname, ext=os.path.splitext(filename)
    
    if "." in newname:
        ext = get_ext(newname) + ext
    return(ext)

In [ ]:
def gauss_mm2sigma(mm):
    return mm/2.35482004503

In [ ]:
def which(list, x=True):
    iter=0
    coords=[]
    
    for elem in list:
        if elem == x:
            coords.append(iter)
        iter+=1
    return(coords)

In [ ]:
def gzip_file(filename, rm_orig=True):
    with open(filename, 'rb') as f_in:
        with gzip.open(filename+'.gz', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
        if rm_orig:
            os.remove(filename)
    return(filename+'.gz')

In [ ]:
def get_key(val, my_dict):
    for key, value in my_dict.items():
         if val == value:
             return key

In [ ]:
def get_step(my_dict, ref_dict, which="prev", ignore=["BASELINE"], give="key"):
    if ignore is None:
        ignore=[]
    if which == "first":
        target=min
    elif which == "prev":
        target=max
    my_keys = list(my_dict.keys())
    for ig in ignore:
        if ig in my_keys:
            pass
        my_keys.remove(ig)
    
    key_val = [ref_dict[key] for key in my_keys]
    if give=="key":
        return get_key(target(key_val), ref_dict)
    elif give=="value":
        return target(key_val)

In [ ]:
def cp_rename(filepath, newpath):
    os.system("cp {} {}".format(filepath, newpath))
    return newpath

# Preprocessing Functions

In [ ]:
def brainroi(img, out_dir):
    roi_img = os.path.join(out_dir, "roi_"+os.path.basename(img))
    os.system("robustfov -i {} -r {}".format(img, roi_img))

    return(roi_img)

In [ ]:
def skullstrip(img, out_dir):
    out_dir  = os.path.join(out_dir, "anat")
    out_file = "brain_" + rm_ext(img)
    skullstr = os.path.join(out_dir, out_file)

    command="bet {} {} -R".format(img, skullstr)
    os.system(command)
    print("       "+command)

    return(skullstr)

In [ ]:
def slicetime(img, out_dir):
    tr=getTR(img)
    tshift_path=os.path.join(out_dir, "func", "t_" + rm_ext(img)+".nii.gz")
    
    command="3dTshift -TR {}s -prefix {} {}".format(tr, tshift_path, img)
    os.system(command)
    print("       "+command)

    return(tshift_path)

In [ ]:
def motcor(img, out_dir):
    motcor_path=os.path.join(out_dir, "func", "m_"+os.path.basename(img))
    _1dfile_path=os.path.join(out_dir, "motion", "1d_"+rm_ext(os.path.basename(img))+".1D")

    command="3dvolreg -base 0 -prefix {} -1Dfile {} {}".format(motcor_path, _1dfile_path, img)
    os.system(command)
    print("       "+command)
    afni2nifti(motcor_path)

    return(motcor_path, _1dfile_path)

In [ ]:
# new spatial normalization
def spatnorm(f_img, a_img, template, out_dir):
    import os

    # lin warp func to struct
    print("       + Linear-warping functional to structural...")
    l_func_omat=os.path.join(out_dir, "spat_norm", "func2str.mat")
    command="flirt -ref {} -in {} -omat {} -dof 6".format(a_img, f_img, l_func_omat)
    os.system(command)
    print("         "+command)
    
    # lin warp struct to template
    print("       + Linear-warping structural to standard template...")
    l_anat_img =os.path.join(out_dir, "anat", "l_" + os.path.basename(a_img))
    l_anat_omat=os.path.join(out_dir, "spat_norm", "aff_str2std.mat")
    command="flirt -ref {} -in {} -omat {} -out {}".format(template, a_img, l_anat_omat, l_anat_img)
    os.system(command)
    print("         "+command)
    
    # non-lin warp struct to template
    print("       + Non-linear-warping structural to standard template...")
    nl_anat_img  =os.path.join(out_dir, "anat", "n" + os.path.basename(l_anat_img))
    cout_anat_img=os.path.join(out_dir, "anat", "cout_" + os.path.basename(nl_anat_img))
    command="fnirt --ref={} --in={} --aff={} --iout={} --cout={} --subsamp=2,2,2,1".format(template, a_img, l_anat_omat, nl_anat_img, cout_anat_img)
    os.system(command)
    print("         "+command)
    
    # make binary mask from non-lin warped image
    print("       + Creating binary mask from non-linearly warped image...")
    bin_nl_anat_img=os.path.join(out_dir, "anat", "bin_" + os.path.basename(nl_anat_img))
    command="fslmaths {} -bin {}".format(nl_anat_img, bin_nl_anat_img)
    os.system(command)
    print("         "+command)
    
    # apply std warp to func data
    print("       + Applying standardized warp to functional data...")
    nl_func_img=os.path.join(out_dir, "func", "nl_"+os.path.basename(f_img))
    command="applywarp --ref={} --in={} --out={} --warp={} --premat={}".format(template, f_img, nl_func_img, cout_anat_img, l_func_omat)
    os.system(command)
    print("         "+command)

    # create tempalte mask
    print("       + Creating binary template mask...")
    mask_path=os.path.join(out_dir, "anat", "mask_" + os.path.basename(template))
    command="fslmaths {} -bin {}".format(template, mask_path)
    os.system(command)
    print("         "+command)

    return(nl_anat_img, nl_func_img, cout_anat_img, l_anat_omat) # anat, func, warp, premat

In [ ]:
def applywarp(in_img, out_img, ref_img, warp_img, premat):
    os.system("fnirt --ref={} --in={} --aff={} --iout={} --cout={} --subsamp=2,2,2,1".format(ref_img, in_img, premat, out_img, warp_img))
    return(out_img)

In [ ]:
def segment(img, out_dir=None):
    if out_dir is None:
        out_dir = os.path.dirname(img)
    seg_path = os.path.join(out_dir, "seg")
    command="fast -n 3 -t 1 -o '{}' '{}'".format(seg_path, img)
    os.system(command)
    print("       "+command)

    return(seg_path)

In [ ]:
def bin_mask(mask, thr=0.5):
    output = os.path.join(os.path.dirname(mask), "bin_"+os.path.basename(mask))
    command="fslmaths {} -thr {} -bin {}".format(mask, thr, output)
    os.system(command)
    
    return(output)

In [ ]:
def roi_tcourse(img, mask, save_path):
    # compute the mean time course for ROI
    command="fslmeants -i '{}' -m '{}' -o '{}'".format(img, mask, save_path)
    os.system(command)
    print("       "+command)
    
    return(save_path)

In [ ]:
def spat_smooth(img, mm, out_dir): 
    sigma=gauss_mm2sigma(mm)
    s_img=os.path.join(out_dir, "s_"+os.path.basename(img))
    
    command="fslmaths {} -kernel gauss {} -fmean {}".format(img, sigma, s_img)
    os.system(command)
    print("       "+command)
    
    return(s_img)

In [ ]:
def combine_nuis(nuis1, nuis2, output):
    if nuis1 is None:
        nuis2_pd=pd.read_csv(nuis2, sep="\s+", header=None)
        nuis2_pd.to_csv(output, sep="\t", header=None, index=False)
        return(output)
    elif nuis2 is None:
        nuis1_pd=pd.read_csv(nuis1, sep="\s+", header=None)
        nuis1_pd.to_csv(output, sep="\t", header=None, index=False)
        return(output)

    nuis1_pd=pd.read_csv(nuis1, sep="\s+", header=None)
    nuis2_pd=pd.read_csv(nuis2, sep="\s+", header=None)

    nuis_cbind = pd.concat([nuis1_pd, nuis2_pd], axis=1)
    nuis_cbind.to_csv(output, sep="\t", header=None, index=False)

    return(output)

In [ ]:
def nuis_reg(img, _1d, out_dir, pref="nuis", poly="1"):

    clean_img=os.path.join(out_dir, pref + "_" + rm_ext(img) + ".nii.gz")
    command="3dDeconvolve -input {} -ortvec  {} {} -polort {} -errts {}".format(img, _1d, pref, poly, clean_img)
    os.system(command)
    print("       "+command)

    return(clean_img)

In [ ]:
def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1

In [ ]:
def meantsBOLD(img, outdir, nomoco):
    dvars_txt = os.path.join(outdir, "dvars.1D")
    dvars_png = os.path.join(outdir, "dvars.png")

    out_compound = os.path.join(outdir, "dvars_outCols.1D")
    out_outliers = os.path.join(outdir, "dvars_outliers.1D")

    command="fsl_motion_outliers -i {} -o {} -s {} -p {} --dvars".format(img, out_compound, dvars_txt, dvars_png)
    if nomoco:
        command+=" --nomoco"

    os.system(command)

    sleep(10)
    if not os.path.exists(out_compound):
        nlines=file_len(dvars_txt)
        dvars_compound=np.zeros([nlines,1], dtype=int)
        dvars_compound=pd.DataFrame(dvars_compound)
        dvars_compound.to_csv(out_compound, sep="\t", index=False, header=False)

    dvars_out=pd.read_csv(out_compound, delim_whitespace=True, header=None)
    dvars_out=dvars_out.sum(axis=1)
    dvars_out=dvars_out.astype("int")
    dvars_out.to_csv(out_outliers, sep="\t", index=False, header=False)

    return(out_outliers)

In [ ]:
def fd_out(img, voxel_size, outdir):
    thresh = voxel_size/2
    fd_txt = os.path.join(outdir, "fd.1D")
    fd_png = os.path.join(outdir, "fd.png")

    out_compound = os.path.join(outdir, "fd_outCols.1D")
    out_outliers = os.path.join(outdir, "fd_outliers.1D")

    command = "fsl_motion_outliers -i {} -o {} -s {} -p {} --fd --thresh={}".format(img, out_compound, fd_txt, fd_png, thresh)
    os.system(command)

    sleep(10)
    if not os.path.exists(out_compound):
        nlines=file_len(fd_txt)
        fd_compound=np.zeros([nlines,1], dtype=int)
        fd_compound=pd.DataFrame(fd_compound)
        fd_compound.to_csv(out_compound, sep="\t", index=False, header=False)

    fd_out=pd.read_csv(out_compound, delim_whitespace=True, header=None)
    fd_out=fd_out.sum(axis=1)
    fd_out=fd_out.astype("int")

    fd_out.to_csv(out_outliers, sep="\t", index=False, header=False)

    return(out_outliers)

In [ ]:
def mk_outliers(dvars, fd, out_dir, method="UNION"):
    if fd is not None:
        fd_mat = pd.read_csv(fd, delimiter="\t", header=None)
    if dvars is not None:
        dvars_mat = pd.read_csv(dvars, delimiter="\t", header=None)
    
    if method=="UNION":
        outliers_mat = (fd_mat + dvars_mat).astype("bool")
        outliers_mat = outliers_mat.astype("int")
    elif method=="INTERSECT":
        outliers_mat = dvars_mat[0]*fd_mat[0]
        outliers_mat = outliers_mat.dropna()
    elif method=="FD":
        outliers_mat = fd_mat
    elif method=="DVARS":
        outliers_mat = dvars_mat
    
    outlier_path = os.path.join(out_dir, "scrub_outliers.txt")
    outliers_mat.to_csv(outlier_path, sep="\t", index=False, header=False)

    return(outlier_path)


In [ ]:
def interp_time(img, out, int_ind, return_full=False):
    if len(img.shape)!=4:
        raise Exception("Nifti image must have 4 dimensions.")
    
    # make blank image
    int_dim = list(img.shape[0:3])
    int_dim.append(int_ind[1]-int_ind[0]-1)
    int_img = np.empty(int_dim)

    for row in range(img.shape[0]):
        for col in range(img.shape[1]):
            for slice in range(img.shape[2]):
                int_val = interp_pts(img[row, col, slice, int_ind[0]], img[row, col, slice, int_ind[1]], int_ind[1]-int_ind[0]-1)
                int_img[row, col, slice, :] = int_val

    if return_full:
        img[:,:,:,out] = int_img
        return(img)
    else:
        return(int_img)


In [ ]:
def interp_pts(start, end, nvals, round=True):
    step_len =(end-start)/(nvals+1)
    interp_set=[start+(step_len*i) for i in range(1,nvals+1)]

    if round:
        interp_set=round_dec(interp_set, [start, end])

    return(interp_set)

In [ ]:
def round_dec(vals, ref):
    n_places=max([n_dec(i) for i in ref])
    rnd_vals=[round(i, n_places) for i in vals]

    return(rnd_vals)

In [ ]:
def n_dec(val):
    n_places=abs(decimal.Decimal(str(val)).as_tuple().exponent)
    return(n_places)

In [ ]:
def surrounding_timepoints(t_range, ind):   
    sub_ind = np.empty((0,2), dtype=int, order='C')
    
    for ii in ind:
        t_start = ii-1
        t_end   = ii+1
        
        while t_start in ind:
            t_start -= 1
        while t_end in ind:
            t_end += 1
        
        if t_start < min(t_range):
            t_start = "NaN"
        if t_end < min(t_range):
            t_end = "NaN"
        
        sub_ind = np.vstack([sub_ind, (t_start, t_end)])
        sub_ind = np.unique(sub_ind, axis=0)
    return(sub_ind)

In [ ]:
def scrubbing(nifti, outliers, out_dir, interpolate=False):
    img = nib.load(nifti)
    mat = img.get_fdata()

    outliers=read_csv(outliers, delimiter="\t", header=None)
    outliers=outliers.values.tolist()
    outliers=which([o[0] for o in outliers], 1)

    print("       + Removing outlier timepoints...")
    mat[:,:,:,outliers]=float("NaN")

    if interpolate:
        print("       + Linearly-interpolating outliers...")
        sub_ind = surrounding_timepoints(range(mat.shape[3]), outliers)

        for oo in range(len(sub_ind)):
            tmp = interp_time(mat, outliers[oo], sub_ind[oo,:], return_full=False)
            mat[:,:,:,(sub_ind[oo,0]+1):sub_ind[oo,1]] = tmp

    new_img = nib.Nifti1Image(mat, img.affine, header=img.header)
    print("       + Saving scrubbed timeseries...")
    scrub_path=os.path.join(out_dir, "scrub_"+rm_ext(os.path.basename(nifti))+".nii")
    nib.save(new_img, scrub_path)
    
    return(gzip_file(scrub_path))

# Wrappers

In [ ]:
def wrapper_lvl2(input_file, config_file):
    config = interpret_config(config_file)
    input  = interpret_input(input_file)
    nrow   = len(input.index)
    
    for row in range(nrow):
        wrapper_lvl1(input.loc[row,:], config)
        print("")
    return(input, config)

In [ ]:
def wrapper_lvl1(input, config):
    create_dirstruct(input["OUTPUT"])

    # logging the print statements
    olog = os.path.join(input["OUTPUT"], "stdout.log")
    open(olog, "w").close()
    sys.stdout = open(olog, "w")

    # copy files to new locations
    cur_func=cp_rename(os.path.join(input["OUTPUT"], "func", "func"   + get_ext(input["FUNC"])), input["FUNC"])
    cur_anat=cp_rename(os.path.join(input["OUTPUT"], "anat", "MPRage" + get_ext(input["FUNC"])), input["ANAT"])

    # create empty dictionary
    pipe_steps={}

    # add voxel size
    voxel_sz=voxel_size(cur_func, excl_time=False)

    # sort step keys by value
    sorted_steps = sorted(step_order.items(), key=operator.itemgetter(1))

    for step_key in sorted_steps:
        step_key = step_key[0]

        # add baseline
        if step_key=="BASELINE":
            pipe_steps={"BASELINE":{"func":cur_func, "anat":cur_anat, "voxel_size":voxel_sz}}
        
        elif step_key=="SKULLSTRIP" and config["SKULLSTRIP"]=="1": 
            # will always run on the baseline
            print("SKULL STRIPPING")
            
            step = get_step(pipe_steps, step_order, which="prev", ignore=None)
            cur_anat = skullstrip(pipe_steps["BASELINE"]["anat"], input["OUTPUT"])
            pipe_steps["SKULLSTRIP"] = {"anat":cur_anat, "func":pipe_steps["BASELINE"]["anat"]}
        
        elif step_key=="SLICETIME" and config["SLICETIME"]=="1":
            print("SLICETIME CORRECTION")

            step = get_step(pipe_steps, step_order, which="prev", ignore=None)
            cur_func = slicetime(cur_func, input["OUTPUT"])
            pipe_steps["SLICETIME"] = {"anat":pipe_steps[step]["anat"], "func":cur_func}
        
        elif step_key=="MOTCOR" and config["MOTCOR"]=="1":
            print("MOTION CORRECTION")

            step = get_step(pipe_steps, step_order, which="prev", ignore=None)
            cur_func, _1dfile_path=motcor(pipe_steps[step]["func"], input["OUTPUT"])
            pipe_steps["MOTCOR"] = {"anat":pipe_steps[step]["anat"], "func":cur_func, "mot_estim":_1dfile_path}
        
        elif step_key=="NORM" and config["NORM"]=="1":
            print("SPATIAL NORMALIZATION")

            step = get_step(pipe_steps, step_order, which="prev", ignore=None)
            cur_anat, cur_func, nl_warp, nl_premat = spatnorm(pipe_steps[step]["func"], pipe_steps[step]["anat"], config["TEMPLATE"], input["OUTPUT"])        
            pipe_steps["NORM"]={"anat":cur_anat, "func":cur_func, "nl_warp":nl_warp, "nl_premat":nl_premat}
        
        elif step_key=="NUISANCE" and config["NUISANCE"]!="0":
            print("NUISANCE SIGNAL REGRESSION")
            
            step = get_step(pipe_steps, step_order, which="prev", ignore=None)
            nuis_path=os.path.join(input["OUTPUT"], "motion", "nuisance_regressors.1D")

            if config["GSR"]=="1":
                print("       + Global Signal Regression...")
                
                step = get_step(pipe_steps, step_order, which="prev", ignore=None)
                csf_mask=segment(pipe_steps[step]["anat"], os.path.join(input["OUTPUT"], "anat", "segment")) + "_pve_0.nii.gz"
                bin_csf_mask=bin_mask(csf_mask, 0.75)
                
                gs_tcourse_path=os.path.join(input["OUTPUT"], "motion", "global_signal.1D")
                gs_tcourse=roi_tcourse(pipe_steps[step]["func"], bin_csf_mask, gs_tcourse_path)
            else:
                gs_tcourse=None
                bin_csf_mask=None

            if config["MOTREG"]=="1":
                print("       + Motion Parameter Regression...")
                mot_tcourse=pipe_steps["MOTCOR"]["mot_estim"]
            else:
                mot_tcourse=None

            if mot_tcourse is None and gs_tcourse is None:
                raise ValueError("Must enable at least one of the following to perform nuisance regression: MOTCOR, GSR")
            
            nuis_tab=combine_nuis(mot_tcourse, gs_tcourse, nuis_path)
            cur_func=nuis_reg(pipe_steps[step]["func"], nuis_tab, os.path.join(input["OUTPUT"],"func"), pref="nuis",poly=config["NUISANCE"])

            pipe_steps["NUISANCE"]={"anat":pipe_steps[step]["anat"], "func":cur_func, "csf_mask":bin_csf_mask, "gsr":gs_tcourse, "mot_reg":mot_tcourse, "nuis_reg":nuis_tab}    

        elif step_key=="SMOOTH" and float(config["SMOOTH"]) > 0:
            print("SPATIAL SMOOTHING")

            step = get_step(pipe_steps, step_order, which="prev", ignore=None)
            cur_func=spat_smooth(pipe_steps[step]["func"], float(config["SMOOTH"]), os.path.join(input["OUTPUT"], "func"))
            pipe_steps["SMOOTH"] = {"anat":pipe_steps[step]["anat"], "func":cur_func}
        
        elif step_key=="SCRUB" and config["SCRUB"]!="NONE":
            print("SCRUBBING fMRI TIME SERIES")
            step = get_step(pipe_steps, step_order, which="prev", ignore=None)

            if config["SCRUB"] in ["UNION", "INTERSECT", "FD"]:
                print("       + Frame-wise Displacement...")
                fd_outliers = fd_out(img=pipe_steps["BASELINE"]["func"], voxel_size=sum(voxel_sz)/len(voxel_sz), outdir=os.path.join(input["OUTPUT"], "motion"))
            else:
                fd_outliers = None

            if config["SCRUB"] in ["UNION", "INTERSECT", "DVARS"]:
                print("       + DVARS...")
                nomoco = config["MOTCOR"]=="1"
                if nomoco:
                    cur_func=pipe_steps["MOTCOR"]["func"]
                else:
                    cur_func=pipe_steps["BASELINE"]["func"]
                dvar_outliers = meantsBOLD(cur_func, os.path.join(input["OUTPUT"], "motion"), nomoco)
            else:
                dvars_outliers = None

            scrub_outliers = mk_outliers(dvar_outliers, fd_outliers, os.path.join(input["OUTPUT"], "motion"), method=config["SCRUB"])
            scrubbed_nifti = scrubbing(pipe_steps[step]["func"], scrub_outliers, os.path.join(input["OUTPUT"], "func"), interpolate=True)

            pipe_steps["SCRUB"] = {"anat":pipe_steps[step]["anat"], "func":scrubbed_nifti, "scrub_outliers":scrub_outliers, "fd":fd_outliers, "dvars":dvar_outliers, "method":config["SCRUB"]}

    sys.stdout.close() 

# Quality Control

In [ ]:
def ica(img, output, n_ic, mask=None):
    ica_img = os.path.join(output, "IC_"+os.path.basename(img))

    if mask is None:
        canica=CanICA(n_components=n_ic, memory_level=0, random_state=0, n_jobs=1, n_init=10)
    else:
        canica=CanICA(n_components=n_ic, memory_level=0, mask=mask, random_state=0, n_jobs=1, n_init=10)

    print("INDEPENDENT COMPONENTS ANALYSIS")
    canica.fit(img)

    canica.components_img_.to_filename(ica_img)
    
    ica_fig_ls = []

    for i, cur_img in enumerate(iter_img(canica.components_img_)):
        ica_jpg = os.path.join(output, "IC"+str(i)+"_"+rm_ext(os.path.basename(img))+".jpg")
        plot_stat_map(cur_img, display_mode="ortho", title="IC %d" % i, colorbar=True, output_file=ica_jpg)
        ica_fig_ls.append(ica_jpg)

    # add in plotting feature

In [ ]:
def nuis_correl(img, nuis, outdir, labels=None, mask=None):
    if mask is not None:
        tcourse=roi_tcourse(img, mask, os.path.join(outdir, "brain_tcourse.1D"))
        tcourse=pd.read_csv(tcourse, sep="\s+", header=None)
    else:
        tcourse=nib.load(img)
        if len(tcourse.shape) != 4:
            raise Exception("The nifti file provided must be 4D.")
        tcourse=tcourse.get_fdata()
        for ii in range(3):
            tcourse=np.mean(tcourse, axis=0)

    # import nuisance variables
    nuis_mat=pd.read_csv(nuis, sep="\s+", header=None)

    # computing the time course/motion parameter correlation
    nuis_correls=np.corrcoef(x=tcourse, y=nuis_mat, rowvar=False)[1:,0]

    return(nuis_correls)

In [ ]:
def qc_preproc(pipe_steps, input, config):
    step_fin = get_step(pipe_steps, step_order, which="prev", ignore=None)

    # ICA
    if config["NORM"]=="1":
        ica_path = ica(pipe_steps[step_fin]["func"], os.path.join(input["OUTPUT"], "quality_control"), int(config["IC"]), pipe_steps["NORM"][mask])
    else:
        ica_path = ica(pipe_steps[step_fin]["func"], os.path.join(input["OUTPUT"], "quality_control"), int(config["IC"]))